Connecting to Google Drive 

In [1]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
!pip install torchmetrics
import import_ipynb 
connect_to_drive()
%cd '/content/gdrive/My Drive/Bioinformatics2'

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=17ce131551550a589ddf3e763f690e969f98f22afffb44a33493b91bf7151811
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
     |████████████████████████████████| 329 kB 5.3 MB/s 
Mounted at /content/gdrive
/content/gdrive/My Drive/Bioinformatics2


Importing modules

In [4]:
import torch
from utils import *
install_requirements()
from VascularDataset import *
import albumentations as A
from torchvision import transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.augmentations.transforms import *
import segmentation_models_pytorch as smp
import wandb
from torchmetrics.functional import iou as iou_torch
from tqdm import tqdm
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR
from torchvision.ops.focal_loss import sigmoid_focal_loss
from segmentation_models_pytorch.losses import FocalLoss

     |████████████████████████████████| 87 kB 3.3 MB/s 
     |████████████████████████████████| 376 kB 16.1 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=802fbcf76acf3fecc5b5275d297f76c1333a045ee15222c7cb8426798fac4b73
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=40046b744d83fab4f1bdcc9e1ff9e14d6193a8c4814ab4e64bbde7c6792dae1e
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 1.7 MB 5.5 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 180 kB 46.0 MB/s 
     |████████████████████████████████| 

Retrieve images paths from txt

In [5]:
input_path_training='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Train'
input_path_test='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Test'

save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

txt_with_training_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_inputs.txt'
txt_with_training_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_targets.txt'
txt_with_validation_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs.txt'
txt_with_validation_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets.txt'
txt_with_test_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_inputs.txt'
txt_with_test_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_targets.txt'

imgs_training_input_split=read_list_images_path_from_txt(txt_with_training_paths_inputs)
imgs_training_target_split=read_list_images_path_from_txt(txt_with_training_paths_targets)
imgs_validation_input_split=read_list_images_path_from_txt(txt_with_validation_paths_inputs)
imgs_validation_target_split=read_list_images_path_from_txt(txt_with_validation_paths_targets)
imgs_test_input=read_list_images_path_from_txt(txt_with_test_paths_inputs)
imgs_test_target=read_list_images_path_from_txt(txt_with_test_paths_targets)

Defining transformations

In [6]:
affine_transform=A.Compose([Flip(), ShiftScaleRotate()])
elastic_transform=A.Compose([ElasticTransform()])
pixel_wise_transform=A.Compose([GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
affine_elastic_pixelwise_transform=A.Compose([Flip(), ShiftScaleRotate(), ElasticTransform(), GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
slice_length=1000
resize_transform=A.Compose([RandomCrop(slice_length, slice_length)])

Defining model, loss function, metrics, and initializing Wandb

In [ ]:
model_name='Unet'
batch_size=10
lr=0.3
momentum=0.9
max_n_epochs=80
encoder_name='timm-resnest101e'
dataset_type='Standard'
step_size=8
gamma=0.9
transforms=affine_elastic_pixelwise_transform

model = smp.Unet(
    encoder_name=encoder_name,       
    encoder_weights='imagenet', 
    in_channels=3,
    activation='sigmoid',                 
)

def dice_loss(pred,target):
    numerator = 2 * torch.sum(pred * target)
    denominator = torch.sum(pred + target)
    return 1 - (numerator + 1) / (denominator + 1)

model.cuda()
optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
#optimizer = Adam(model.parameters(), lr=lr)
loss=dice_loss
#loss=FocalLoss(mode='binary',alpha=0.3)
scheduler=StepLR(optimizer, step_size=step_size, gamma=gamma)

wandb.init(project="Bioinformatics_segmentation_experiments_Unet++",
           config={
              "batch_size": batch_size,
              "model": model_name,
              "learning_rate": lr,
              "encoder_name": encoder_name,
              "max_n_epochs": max_n_epochs,
              "momentum": momentum,
              "optimizer": optimizer,
              "dataset": dataset_type,
              "loss": loss,
              "metrics": iou_torch,
              "transforms": transforms,
              "scheduler": scheduler,
              "step_size":step_size,
              "gamma":gamma
           })

run_id=wandb.run.id
save_model_path=save_model_folder+run_id

wandb: Currently logged in as: emafasce (use `wandb login --relogin` to force relogin)


Defining dataset and dataloaders

In [ ]:
training_dataset=VascularDataset(imgs_training_input_split, imgs_training_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685), transform=transforms)
validation_dataset=VascularDataset(imgs_validation_input_split, imgs_validation_target_split, mean_normalization=(0.7160, 0.4878, 0.6433), std_normalization=(0.1680, 0.2098, 0.1685))
test_dataset=VascularDataset(imgs_test_input, imgs_test_target)

save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

print(f'Training dataset length: {len(training_dataset)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

train_dataloader=DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_dataloader=DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Training dataset length: 160
Validation dataset length: 40
Test dataset length: 10


Training, validation and logging into Wandb

In [ ]:
lowest_validation_loss=float('inf')

for epoch in range(1, max_n_epochs+1, 1):

  model.train()
  training_loss=0
  training_iou=0
  training_samples=0

  for x,y in tqdm(train_dataloader): 
    optimizer.zero_grad()
    out=model(x.cuda().float())
    myloss=loss(out, y.cuda().float())
    #myloss=sigmoid_focal_loss(out, y.cuda().long(), alpha=0.3)
    print(myloss)

    for out_i,y_i in zip(out,y.cuda()):
      iou=iou_torch(out_i.detach(),y_i.int())
      training_iou+=iou
      training_samples+=1

    myloss.backward()
    optimizer.step()
    training_loss+=myloss.item()
  
  print(f'Epoch #{epoch}, training loss: {training_loss/training_samples}, training iou: {training_iou/training_samples}')

  scheduler.step()

  model.eval()
  validation_loss=0
  validation_iou=0
  validation_samples=0

  for x,y in tqdm(valid_dataloader): 
    out=model(x.cuda())
    myloss=loss(out.float(), y.cuda().float())
    #myloss=sigmoid_focal_loss(out, y.cuda().long(), alpha=0.3)

    for out_i,y_i in zip(out,y.cuda()):
      iou=iou_torch(out_i.detach(),y_i.int())
      validation_iou+=iou
      validation_samples+=1

    validation_loss+=myloss.item()

  print(f'Epoch #{epoch}, validation loss: {validation_loss/validation_samples}, validation iou: {validation_iou/validation_samples}')

  if validation_loss/validation_samples<lowest_validation_loss:
    lowest_validation_loss=validation_loss/validation_samples
    torch.save(model,f'{save_model_path}.pth')
    last_epoch_saved=epoch

  wandb.log({"training loss": training_loss/training_samples, "validation loss": validation_loss/validation_samples, "epoch":epoch})
  wandb.log({"training iou": training_iou/training_samples, "validation iou": validation_iou/validation_samples, "epoch": epoch})

print(f'Last epoch saved: {last_epoch_saved}')
wandb.log({"last epoch saved": last_epoch_saved})

  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


tensor(0.6183, device='cuda:0', grad_fn=<RsubBackward1>)


  6%|▋         | 1/16 [00:07<01:51,  7.46s/it]

tensor(0.6177, device='cuda:0', grad_fn=<RsubBackward1>)


 12%|█▎        | 2/16 [00:08<00:52,  3.72s/it]

...

tensor(0.2812, device='cuda:0', grad_fn=<RsubBackward1>)


100%|██████████| 16/16 [00:53<00:00,  3.36s/it]


Epoch #80, training loss: 0.028621387481689454, training iou: 0.6415478587150574


100%|██████████| 4/4 [00:11<00:00,  2.85s/it]

Epoch #80, validation loss: 0.027398622035980223, validation iou: 0.6499817371368408
Last epoch saved: 68
